# Open Demand

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import pyodbc # SQL Connection
import sqlCredentials as sql

## Import Database

In [ ]:
#BookXCenterProduction
prod_db = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'Server=52.86.56.66;'
    'Database=BookXCenterProduction;'
    'UID='+sql.username+';'
    'PWD='+sql.password+';'
    'Trusted_connection=no;'
)

## Define clean_up

In [ ]:
def clean_up(phrase):
    phrase = phrase.lstrip()
    phrase = phrase.rstrip()
    phrase = phrase.upper()
    return phrase

## Import Tables

In [ ]:
# Inventory
inventory_sql = """
SELECT 
    ItemCode AS isbn
    , WhsCode AS po_whse
    , AVG(OnHand) AS Onhand
FROM 
	SAP.InventoryReportView
WHERE
	WhsCode IN ('WW', 'AW')
	AND OnHand > 0
GROUP BY
	ItemCode
	, WhsCode
"""

# Line Purchase Order
PO_sql = """
WITH PO AS
(SELECT 
	CASE
		WHEN Rate = 0 THEN 1
		ELSE Rate
	END AS EXCHANGE
	, ISBN
	, OpenQuantity
	, GrossPrice
	, [Discount]
	, ShippingCostUSD
	, Whse
    , [RowDeliveryDate]
    , [DocumentNumber]
    , [DocumentName]
    , [DocumentStatus]
    , [BPCode]
    , [BPName]
    , [OrderStatus]
FROM [SAP].[LinePurchaseOrderView]
)

SELECT
	[ISBN]
	, OpenQuantity
	, (GrossPrice*EXCHANGE*(1-[Discount]/100) +ShippingCostUSD) AS LandedCost
	, [Whse]
    , [RowDeliveryDate]
    , [DocumentNumber]
    , [DocumentName]
    , [BPCode]
    , [BPName]
    , [OrderStatus]
	, (CASE
		WHEN [RowDeliveryDate] > GETDATE() THEN 'OnTime'
		ELSE 'Late'
	END) AS DeliveryStatus
FROM 
	PO
WHERE 
	Whse in ('WW', 'TB') 
	AND OpenQuantity > 0
"""
# Line Sales Order

SO_sql = """
SELECT 
	Isbn
	, OpenQuantity
	, UnitPrice AS SalePrice
	, WarehouseCode
	, DeliveryDate
	, DocumentNumber
	, DocumentName
	, CustomerOrVendorCode
	, CustomerOrVendorName
	, Condition
	, (CASE
			WHEN DeliveryDate > GETDATE() THEN 'OnTime'
			ELSE 'Late'
		END) AS DeliveryStatus
FROM
	[SAP].[SaleOrderReportView]
WHERE OpenQuantity > 0 AND WarehouseCode IN ('WW')
"""
receiving_sql = """
WITH RECEIVED AS
(SELECT
	CASE
		WHEN x_Rate = 0 THEN 1
		ELSE x_Rate
	END AS EXCHANGE
	, [ISBN]
	, [Price]
	, [Discount]
	, [ShippingCost]
	, [WarehouseCode]
	, [PostingDate]
    , [DocName]
	, [DocNum]
	, [BPCode]
	, [BPName]
FROM [SAP].[GoodsReceiptReportView]
)

SELECT
	[ISBN]
	, ([Price]*[EXCHANGE]*(1-[Discount]/100) + [ShippingCost]) AS landed_cost
	, [WarehouseCode] AS po_whse
	, [PostingDate] AS received_date
    , [DocNum] AS po_number 
	, [DocName] AS po_name
	, [BPCode] AS supplier_code
	, [BPName] AS supplier
  FROM RECEIVED
  WHERE WarehouseCode = 'WW'
  ORDER BY PostingDate desc
  """



In [ ]:
inventory = pd.read_sql(inventory_sql, prod_db)
inventory.columns = map(str.lower, inventory.columns)
for col in ['isbn', 'po_whse']:
    inventory[col] = inventory.apply(lambda x: clean_up(x[col]), axis =1)
inventory.rename(columns={'itemcode': 'isbn'}, inplace = True)
print(inventory.isna().sum())
inventory.head()

In [ ]:
receiving = pd.read_sql(receiving_sql, prod_db)
receiving.dropna(inplace=True)
receiving.columns = map(str.lower, receiving.columns)
receiving['isbn'] = receiving['isbn'].astype(str)
for col in ['isbn', 'po_whse', 'po_name', 'supplier_code', 'supplier']:
    receiving[col] = receiving.apply(lambda x: clean_up(x[col]), axis =1)
print(receiving.isna().sum())
print(receiving.dtypes)
receiving.head()

In [ ]:
inventory_ww = inventory[inventory['po_whse'] == 'WW']
inventory_ww = inventory_ww.merge(receiving, how='left', on=['isbn', 'po_whse'] )
inventory_ww.sort_values('received_date', ascending=False, inplace=True, ignore_index=True)
inventory_ww = inventory_ww.groupby(['isbn'])['onhand', 'landed_cost', 'po_whse','received_date', 'po_number', 'po_name', 'supplier_code', 'supplier'].first().reset_index()
inventory_ww.head()

In [ ]:
po = pd.read_sql(PO_sql, prod_db)
po.columns = map(str.lower, po.columns)
po.rename(columns={'bpname': 'supplier', 'bpcode': 'supplier_code', 'openquantity': 'po_open_qty', 'whse': 'po_whse', 'documentnumber' : 'po_number', 'documentname': ' po_name', 'deliverystatus': 'po_date_status'}, inplace = True)
for col in ['isbn', 'po_whse', ' po_name', 'supplier_code', 'supplier', 'orderstatus', 'po_date_status']:
    po[col] = po.apply(lambda x: clean_up(x[col]), axis =1)
print(po.isna().sum())
po.sort_values(['rowdeliverydate', 'po_whse', 'landedcost'], ascending=[True,False,True], inplace=True, ignore_index=True)
po.head()

In [ ]:
so = pd.read_sql(SO_sql, prod_db)
so.columns = map(str.lower, so.columns)
so.rename(columns={'openquantity': 'so_open_qty','warehousecode':'so_whse','customerorvendorname': 'customer', 'customerorvendorcode': 'customer_code', 'deliverydate' : 'customer_drop_date', 'documentnumber': 'so_number', 'documentname': 'so_name','deliverystatus': 'so_date_status'}, inplace = True)
for col in ['isbn', 'so_whse', 'so_name', 'customer', 'customer_code', 'condition', 'so_date_status']:
    so[col] = so.apply(lambda x: clean_up(x[col]), axis =1)
print(so.isna().sum())
so.sort_values(['saleprice', 'customer_drop_date'], ascending=[False,True], inplace=True, ignore_index=True)
so.head()

In [ ]:
so.columns

In [ ]:
#ALLOCATE RECEIVED inventory_ww INTO THE 
inv_so = pd.DataFrame()
inventory_ww.reset_index(inplace = True, drop = True)
for so_id in list(so[so['isbn'].isin(list(inventory_ww['isbn']))].index.values):
    for inv_id in list(inventory_ww[inventory_ww['isbn'].isin(list(so['isbn']))].index.values):
        if (inventory_ww.iloc[inv_id, :]['isbn'] == so.iloc[so_id, :]['isbn']) and (inventory_ww.iloc[inv_id, :]['onhand'] > 0) and (so.iloc[so_id, :]['so_open_qty'] > 0):
            row_inv_so = pd.merge(inventory_ww.iloc[[inv_id], :], so.iloc[[so_id], :], how='left', on = 'isbn' )
            row_inv_so['allocate_qty'] = row_inv_so[['onhand','so_open_qty']].min( axis = 1, skipna = True)
            inventory_ww.loc[inv_id,'onhand'] = inventory_ww.loc[inv_id,'onhand'] - row_inv_so.iloc[0,:]['allocate_qty']
            so.loc[so_id,'so_open_qty'] = so.loc[so_id,'so_open_qty'] - row_inv_so.iloc[0,:]['allocate_qty']
            row_inv_so['orderstatus'] = 'READY TO SHIP'
            row_inv_so['inventory_ww_date_status'] = so.iloc[so_id, :]['so_date_status']
            inv_so = inv_so.append(row_inv_so)


In [ ]:
inv_so.head()

In [ ]:
# po_so = pd.DataFrame()
# for so_id in list(so[so['isbn'].isin(list(po['isbn'])) & (so['so_open_qty'] > 0)].index.values):
#     for po_id in list(po[po['isbn'].isin(list(so['isbn'])) & (po['po_open_qty'] > 0)].index.values):
#         if (po.iloc[po_id, :]['isbn'] == so.iloc[so_id, :]['isbn']) and (po.iloc[po_id, :]['po_open_qty'] > 0) and (so.iloc[so_id, :]['so_open_qty'] > 0) and ((so.iloc[so_id, :]['saleprice'] - po.iloc[po_id, :]['landedcost']) >= 0) & (so.iloc[so_id, :]['customer_drop_date'] > po.iloc[po_id, :]['rowdeliverydate']): #ALLOCATE OPEN PO ON TIME FOR THE SO WITH POSITIVE COMMISSION
#             row_po_so_good = pd.merge(po.iloc[[po_id], :], so.iloc[[so_id], :], how='left', on = 'isbn' )
#             row_po_so_good['qty'] = row_po_so_good[['po_open_qty','so_open_qty']].min( axis = 1, skipna = True)
#             po.loc[po_id,'po_open_qty'] = po.loc[po_id,'po_open_qty'] - row_po_so_good.iloc[0,:]['qty']
#             so.loc[so_id,'so_open_qty'] = so.loc[so_id,'so_open_qty'] - row_po_so_good.iloc[0,:]['qty']
#             row_po_so_good['time_status'] = so.iloc[so_id, :]['so_date_status']
#             # if (so.iloc[so_id, :]['so_date_status'] == 'LATE') & (po.iloc[po_id, :]['po_date_status'] == 'LATE'):
#             #     row_po_so_good['action'] = 'UPDATE PO DATE, UPDATE SO DATE'
#             # elif (so.iloc[so_id, :]['so_date_status'] == 'ONTIME') & (po.iloc[po_id, :]['po_date_status'] == 'LATE'):
#             #     row_po_so_good['action'] = 'UPDATE PO DATE'
#             # elif  (so.iloc[so_id, :]['so_date_status'] == 'LATE') & (po.iloc[po_id, :]['po_date_status'] == 'ONTIME'):
#             #     row_po_so_good['action'] = 'UPDATE PO DATE'
#             # else:
#             #     row_po_so_good['action'] = 'PROFIT'
#         elif (po.iloc[po_id, :]['isbn'] == so.iloc[so_id, :]['isbn']) and (po.iloc[po_id, :]['po_open_qty'] > 0) and (so.iloc[so_id, :]['so_open_qty'] > 0) and ((so.iloc[so_id, :]['saleprice'] - po.iloc[po_id, :]['landedcost']) < 0) & (so.iloc[so_id, :]['customer_drop_date'] > po.iloc[po_id, :]['rowdeliverydate']): #ALLOCATE OPEN PO ON TIME FOR THE SO WITH NEGATIVE COMMISSION
#             row_po_so_good = pd.merge(po.iloc[[po_id], :], so.iloc[[so_id], :], how='left', on = 'isbn' )
#             row_po_so_good['qty'] = row_po_so_good[['po_open_qty','so_open_qty']].min( axis = 1, skipna = True)
#             po.loc[po_id,'po_open_qty'] = po.loc[po_id,'po_open_qty'] - row_po_so_good.iloc[0,:]['qty']
#             so.loc[so_id,'so_open_qty'] = so.loc[so_id,'so_open_qty'] - row_po_so_good.iloc[0,:]['qty']
#             row_po_so_good['time_status'] = so.iloc[so_id, :]['so_date_status']
#             # if (so.iloc[so_id, :]['so_date_status'] == 'LATE') & (po.iloc[po_id, :]['po_date_status'] == 'LATE'):
#             #     row_po_so_good['action'] = 'UPDATE PO DATE, UPDATE SO DATE'
#             # elif (so.iloc[so_id, :]['so_date_status'] == 'ONTIME') & (po.iloc[po_id, :]['po_date_status'] == 'LATE'):
#             #     row_po_so_good['action'] = 'UPDATE PO DATE'
#             # elif  (so.iloc[so_id, :]['so_date_status'] == 'LATE') & (po.iloc[po_id, :]['po_date_status'] == 'ONTIME'):
#             #     row_po_so_good['action'] = 'UPDATE PO DATE'
#             # else:
#             #     row_po_so_good['action'] = 'NO PROFIT'
#         po_so = po_so_good.append(row_po_so_good)

In [ ]:
po_so = pd.DataFrame()
for so_id in list(so[so['isbn'].isin(list(po['isbn'])) & (so['so_open_qty'] > 0)].index.values):
    for po_id in list(po[po['isbn'].isin(list(so['isbn'])) & (po['po_open_qty'] > 0)].index.values):
        if (po.iloc[po_id, :]['isbn'] == so.iloc[so_id, :]['isbn']) and (po.iloc[po_id, :]['po_open_qty'] > 0) and ((so.iloc[so_id, :]['so_open_qty'] > 0) and (so.iloc[so_id, :]['saleprice'] - po.iloc[po_id, :]['landedcost']) > -5):
            row_po_so = pd.merge(po.iloc[[po_id], :], so.iloc[[so_id], :], how='left', on = 'isbn' )
            row_po_so['qty'] = row_po_so[['po_open_qty','so_open_qty']].min( axis = 1, skipna = True)
            po.loc[po_id,'po_open_qty'] = po.loc[po_id,'po_open_qty'] - row_po_so.iloc[0,:]['qty']
            so.loc[so_id,'so_open_qty'] = so.loc[so_id,'so_open_qty'] - row_po_so.iloc[0,:]['qty']
            po_so = po_so.append(row_po_so)

In [ ]:
po_so.head()

In [ ]:
"""
RETAIL AW WAREHOUSE"

"""

# inv_aw = inventory[inventory['po_whse'] == 'AW']
# inv_aw.reset_index(inplace = True, drop=True)
# aw_so = pd.DataFrame()
# for so_id in list(so[so['isbn'].isin(list(inv_aw['isbn']))].index.values):
#     for aw_inv_id in list(inv_aw[inv_aw['isbn'].isin(list(so['isbn']))].index.values):
#         if (inv_aw.iloc[aw_inv_id, :]['isbn'] == so.iloc[so_id, :]['isbn']) and (inv_aw.iloc[aw_inv_id, :]['onhand'] > 0) and (so.iloc[so_id, :]['so_open_qty'] > 0):
#             row_aw_so = pd.merge(inv_aw.iloc[[aw_inv_id], :], so.iloc[[so_id], :], how='left', on = 'isbn' )
#             row_aw_so['allocate_qty'] = row_aw_so[['onhand','so_open_qty']].min( axis = 1, skipna = True)
#             inv_aw.loc[aw_inv_id,'onhand'] = inv_aw.loc[aw_inv_id,'onhand'] - row_aw_so.iloc[0,:]['allocate_qty']
#             so.loc[so_id,'so_open_qty'] = so.loc[so_id,'so_open_qty'] - row_aw_so.iloc[0,:]['allocate_qty']
#             row_aw_so['orderstatus'] = 'READY TO SHIP'
#             row_aw_so['inv_aw_date_status'] = so.iloc[so_id, :]['so_date_status']
#             aw_so = aw_so.append(row_aw_so)

In [ ]:
# aw_so.head()

In [ ]:
po.sort_values(['rowdeliverydate', 'po_whse', 'landedcost'], ascending=[True,False,True], inplace=True, ignore_index=True)
so.sort_values(['customer_drop_date', 'saleprice'], ascending=[True,False], inplace=True, ignore_index=True)

In [ ]:
# real_sale = pd.DataFrame()
# real_sale = inv_so.append(po_so_good)

In [ ]:
with pd.ExcelWriter('supply_chain/report.xlsx') as writer:
    inv_so.to_excel(writer,sheet_name = 'to_be_shipped', index= False)
    po[po['po_open_qty'] > 0 ].to_excel(writer, sheet_name = 'po_no_room', index= False)
    so[so['so_open_qty'] > 0 ].to_excel(writer, sheet_name = 'open_so', index= False)
    inventory_ww[inventory_ww['onhand'] > 0 ].to_excel(writer, sheet_name = 'extras_inventory', index= False)
    po_so.to_excel(writer,sheet_name = 'expected_sales', index= False)
    # aw_so.to_excel(writer,sheet_name = 'aw_reallocation', index= False) # Line when trying to make a sale from AW Warehouse.